In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pytz

Código que irá plotar gráficos de porcentagem de ocupações e reservas para cada minuto do dia.

### Lendo e filtrando os dados coletados da API

In [ ]:
# Lendo dados coletados da API
df = pd.read_csv('modo_data.csv', usecols=[0,2,3,5])

In [ ]:
def str_to_datetime(df_time):
    """ 
    Reformatando de string para datetime.
    
    Parameters
    ----------
    df_time : pandas.DataFrame, string
        Dataframe com strings a serem convertidas para datetime.
    
    Returns
    ----------
    date_list : pandas.DataFrame, datetime
        Dataframe com valores em datetime para possíveis fusos de Vancouver.
    
    """
    date_list = []
    
    # Formatos de fuso horário comum de Vancouver e 
    # fuso horário característico de horário de verão
    format_string = ['%Y-%m-%d %H:%M:%S.%f-08:00', '%Y-%m-%d %H:%M:%S.%f-07:00',
                     '%Y-%m-%d %H:%M:%S-08:00', '%Y-%m-%d %H:%M:%S-07:00']
    
    print(datetime.datetime.now())
    for date in df_time:
        for fmt in format_string:
            try:
                date_list.append(datetime.datetime.strptime(str(date), fmt))
                break
            except:
                pass
            
    print(datetime.datetime.now())
    return pd.DataFrame(date_list)

In [ ]:
# Retirando dados nan
df.dropna(axis=0, how='any', inplace=True)

# Convertendo datetime strings para o tipo datetime
df['Capture_time'] = str_to_datetime(df['Capture_time'])

# Colleting vehicle ids
car_ids = list(df.CarID.unique())

In [ ]:
def convert_datetime_timezone(dt, tz1, tz2):
    """
    Converte uma hora no fuso UTC ou São Paulo para um provável fuso de Vancouver.
    
    Parameters
    ------------
    dt : unix timestamp
        Timestamp a ser convertido para outro fuso horário.
    
    tz1, tz2 : Timezone String
        Time zone atual e a que a hora irá ser convertida.
    
    Returns
    ----------
    dt : unix timestamp
        Timestamp já convertida para o fuso de Vancouver.
    
    """    
    
    tz1 = pytz.timezone(tz1)
    tz2 = pytz.timezone(tz2)

    dt = datetime.datetime.fromtimestamp(dt)
    dt = datetime.datetime.strptime(str(dt),"%Y-%m-%d %H:%M:%S")
    dt = tz1.localize(dt)
    dt = dt.astimezone(tz2)
    
    try:
        # Fuso horário comum de Vancouver
        dt = datetime.datetime.strptime(str(dt),"%Y-%m-%d %H:%M:%S-08:00")
    except:
        # Fuso horário característico de horário de verão em Vancouver
        dt = datetime.datetime.strptime(str(dt),"%Y-%m-%d %H:%M:%S-07:00")
    
    dt = int(dt.timestamp())

    return dt

In [ ]:
# Eliminando intervalos de disponibilidade futuros
print ('Size with future interval: '+str(len(df)))

# Separando o dataframe com os intervalos futuros
df_with_future = df
df = pd.DataFrame()

# Ordenando por capture time e pelo start time
df_with_future.sort_values(by=['Capture_time', 'StartTime'], inplace=True)

for car in car_ids:
    # Refazendo o dataframe somente com os intervalos presentes
    # já que a api retorna os intervalos de disponibilidade futuros
    df = df.append(df_with_future[df_with_future['CarID'] == car].drop_duplicates(subset='Capture_time', keep='first'))

print ('Size without future interval: '+str(len(df)))

df.sort_values(by='Capture_time', inplace=True)

# Retirando dados nan
df.dropna(axis=0, how='any', inplace=True)

## Porcentagem de carros ocupados a cada minuto

Realizando a coleta de veículos ocupados a cada minuto dos dados, com o seguinte método:<br>
Se o veículo estiver disponível somente em um intervalo futuro, então ele está ocupado neste minuto analisado.

In [ ]:
in_travel = 0
andando_weekdays = []
andando_weekends = []


# Percorre todo o dataframe para verificar quais carros estão andando em dado minuto
for i in range(1, len(df)):
    capture_time_atual = int(df['Capture_time'].iloc[i].timestamp())

    capture_time_anterior = int(df['Capture_time'].iloc[i-1].timestamp())

    start_time = int(df['StartTime'].iloc[i])

    request_start = df['RequestStart'].iloc[i]

    # Enquanto está no mesmo minuto, é analisado se o carro está andando
    if (capture_time_atual == capture_time_anterior):
        if (start_time > request_start):
            in_travel += 1
    else:
        porcentagem = (in_travel/len(car_ids))*100
        
        # Verifica que a data está entre segunda(1) e sexta(5)
        if (int(datetime.datetime.fromtimestamp(capture_time_anterior).strftime('%w')) > 0 and 
            int(datetime.datetime.fromtimestamp(capture_time_anterior).strftime('%w')) < 6):
            andando_weekdays.append([capture_time_anterior, in_travel, porcentagem])
        else:
            andando_weekends.append([capture_time_anterior, in_travel, porcentagem])
        in_travel = 0
        
dfIn_Travel_weekdays = pd.DataFrame(andando_weekdays, columns=['capture_time', 'total_in_travel', 'percentage'])
dfIn_Travel_weekends = pd.DataFrame(andando_weekends, columns=['capture_time', 'total_in_travel', 'percentage'])

In [ ]:
def from_timestamp_list(timestamp_list):
    
    datetime_list = []
    
    for date in timestamp_list:
        datetime_list.append(datetime.datetime.fromtimestamp(int(date)))
    
    return pd.DataFrame(datetime_list)

In [ ]:
# Formatando os dados de unix timestamp para datetime

dfWeekdays = dfIn_Travel_weekdays

dfWeekdays['capture_time'] = from_timestamp_list(dfWeekdays['capture_time'])    
    
    
dfWeekends = dfIn_Travel_weekends

dfWeekends['capture_time'] = from_timestamp_list(dfWeekends['capture_time'])

In [ ]:
# Salvando dados já processados
dfWeekends.to_csv('weekends_v2.csv', index=False, encoding='utf-8')
dfWeekdays.to_csv('weekdays_v2.csv', index=False, encoding='utf-8')

In [ ]:
# Leitura de dados já processados se necessário

# dfWeekends = pd.read_csv('weekends_v2.csv')
# dfWeekdays = pd.read_csv('weekdays_v2.csv')

# dfWeekdays['capture_time'] = pd.to_datetime(dfWeekdays['capture_time'])
# dfWeekends['capture_time'] = pd.to_datetime(dfWeekends['capture_time'])

In [ ]:
# Plot da porcentagem de carros alocados em dias de semana
plt.plot(dfWeekdays['capture_time'],dfWeekdays['percentage'])
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
# Plot da porcentagem de carros alocados em dias de final de semana
plt.plot(dfWeekends['capture_time'],dfWeekends['percentage'])
plt.gcf().autofmt_xdate()
plt.show()

## Porcentagem média de carros ocupados em cada minuto

Importante função que realiza a porcentagem média para todos os minutos do intervalo de coleta, assim organizando minuto a minuto a porcentagem respectiva.

In [ ]:
def media(df): 
    """
    Faz a media das porcentagens minuto a minuto de todo o dataset.
    
    Parameters
    ------------
    df : Pandas dataframe
        Dados a serem analisados, com uma coluna dos horários e outra com as porcentagens.
    
    Returns
    ----------
    media : Pandas dataframe
        Dados com a média das porcentagens para 24 horas.
    
    """
    minute = []

    # Criando um vetor que irá sinalizar a quantidade de minutos corridos até tal registro
    for i in range(len(df)):
        capture_time = df['capture_time'].iloc[i]
        minute.append(capture_time.minute + (capture_time.hour * 60))

    # Ordenando o dataset por minutos corridos para facilitar a soma de valores
    df['minute'] = minute
    df = df.sort_values(by=['minute', 'capture_time'])

    valores = pd.DataFrame()
    media = []
    for i in range(1,len(df)):
        minute_atual = df['minute'].iloc[i-1]
        minute_proximo = df['minute'].iloc[i]

        # Enquanto está no mesmo valor de minutos corridos os valores percentuais 
        # são armazenados para ser calculada a média de tal minuto no intervalo de 24h
        if (minute_proximo == minute_atual):
            valores = valores.append([df['percentage'].iloc[i-1]])
        else:
            valores = valores.append([df['percentage'].iloc[i-1]])
            media.append([df['capture_time'].iloc[i-1].strftime('%H:%M'), 
                          float(valores.mean()), float(valores.std())])
            valores = pd.DataFrame()

    media = pd.DataFrame(media, columns=['time', 'mean', 'std'])

    # Formatando a hora para datetime
    for i in range(len(media)):
        media['time'].iloc[i] = datetime.datetime.strptime(media['time'].iloc[i], '%H:%M').time()

    return media

In [ ]:
# Fazendo a média das porcentagens de cada dia
dfWeekdays = dfWeekdays.sort_values(by='capture_time')
mediaWeekdays = media(dfWeekdays)

dfWeekends = dfWeekends.sort_values(by='capture_time')
mediaWeekends = media(dfWeekends)

In [ ]:
# Salvando dados já processados
mediaWeekdays.to_csv('mediaWeekdays_v2.csv', index=False, encoding='utf-8')
mediaWeekends.to_csv('mediaWeekends_v2.csv', index=False, encoding='utf-8')

In [ ]:
# Leitura de dados já processados se necessário

# mediaWeekdays = pd.read_csv('mediaWeekdays_v2.csv')
# mediaWeekends = pd.read_csv('mediaWeekends_v2.csv')

In [ ]:
# Ordenando pelo tempo
mediaWeekdays = mediaWeekdays.sort_values(by=['time'])
mediaWeekends = mediaWeekends.sort_values(by=['time'])

## Gráficos de porcentagem média de carros ocupados em cada minuto

In [ ]:
import numpy as np

# Plot da media das porcentagens dos dias de semana
fig, ax = plt.subplots()
# Curva dos carros andando
ax.plot(range(len(mediaWeekdays['time'])),mediaWeekdays['mean'], label='Carros Ocupados')

# Curvas representando o intervalo de desvio padrão
ax.plot(range(len(mediaWeekdays['time'])), mediaWeekdays['mean']+mediaWeekdays['std'], alpha=150, c='gray', label='Desvio Padrão')
ax.plot(range(len(mediaWeekdays['time'])), mediaWeekdays['mean']-mediaWeekdays['std'], alpha=150, c='gray')

# Modificando os labels das horas
ax.xaxis.set_ticks(np.arange(0, 1441, 120))

fig.canvas.draw()

labels = [item.get_text() for item in ax.get_xticklabels()]
labels = range(0,26,2)

ax.set_xticklabels(labels)

# Legendas e label dos eixos
plt.legend(bbox_to_anchor=(0.01, 0.99), loc=2, borderaxespad=0.2)
plt.ylabel('Percentual')
plt.xlabel('Horário')

# Salvando o plot
plt.savefig('Weekdays_v2.pdf', bbox_inches='tight')

plt.show()

In [ ]:
# Plot da media das porcentagens dos dias de semana
fig, ax = plt.subplots()
# Curva dos carros andando
ax.plot(range(len(mediaWeekends['time'])),mediaWeekends['mean'], label='Carros Reservados')

# Curvas representando o intervalo de desvio padrão
ax.plot(range(len(mediaWeekends['time'])), mediaWeekends['mean']+mediaWeekends['std'], alpha=150, c='gray', label='Desvio Padrão')
ax.plot(range(len(mediaWeekends['time'])), mediaWeekends['mean']-mediaWeekends['std'], alpha=150, c='gray')

# Modificando os labels das horas
ax.xaxis.set_ticks(np.arange(0, 1441, 120))

fig.canvas.draw()

labels = [item.get_text() for item in ax.get_xticklabels()]
labels = range(0,26,2)

ax.set_xticklabels(labels)

# Legendas e label dos eixos
plt.legend(bbox_to_anchor=(0.01, 0.99), loc=2, borderaxespad=0.2)
plt.ylabel('Percentual')
plt.xlabel('Horário')

# Salvando o plot
plt.savefig('Weekends_v2.pdf', bbox_inches='tight')

plt.show()

## Extração das porcentagens de carros reservados

In [ ]:
# CSV criado a partir dos dados coletados do arquivo ModoApi_Data_Filter
dfTravels = pd.read_csv('travels_test.csv')

In [ ]:
# Ordenando por id e inicio do tempo de reserva
dfTravels = dfTravels.sort_values(by=['car_id', 'start'])

In [ ]:
# Função desnecessária para o algoritmo atual de contagem de reservas
def concat_consec():
    # Concatenando reservas consecutivas
    # Enquanto estiver com viagens consecutivas percorrer o dataframe
    while (len(dfTravels[dfTravels['only_new_reserves'] == False]) > 0):
        i = 0
        while(i < len(dfTravels)-1):
            # Se a viagem atual for comum e a seguinte consecutiva, analisar a possível concatenação
            if (dfTravels['only_new_reserves'].iloc[i] and 
                not dfTravels['only_new_reserves'].iloc[i+1]):

                # Se a viagem consecutiva terminar depois da viagem anterior efetuar concatenação
                if (dfTravels['end'].iloc[i] < dfTravels['end'].iloc[i+1]):
                    dfTravels['end'].iloc[i] = dfTravels['end'].iloc[i+1]

                # Caso contrario, somente retirar do dataframe a viagem consecutiva
                dfTravels = dfTravels.drop(dfTravels.index[i+1])
                dfTravels.index = range(len(dfTravels))
                i -= 1

            i+=1

    dfTravels.index = range(len(dfTravels))
    dfTravels.to_csv('travels_concat_v2.csv')

###  Função para contar a porcentagem de carros reservados nos dias passados como parâmetro

Realizando a coleta das porcentagens de reservas com o seguinte algoritmo:<br><br>
Inicialmente necessito de lista de todos os minutos de coleta e outra com todos os intervalos de reserva para cada veículo.<br>
A partir disso construimos uma timeline para cada carro, percorrendo todos os minutos de coleta. A timeline é montada analisando se no minuto atual existe alguma reserva que o engloba, assim é adicionado um inteiro para representar que ele estava reservado em tal minuto. Assim realizo a soma das timelines de cada veículo e calculo a média pela quantidade de veículos, para cada minuto.
>Nota: Tal algoritmo não é o mais otimizado, porém torna o método mais intuítivo e com uma menor chance de ocorrer erros.<br>
A forma mais otimizada seria percorrer todos os minutos e verificar quantas reservas registradas englobam tal minuto, porém tal método pode gerar erros se não adaptado corretamente ao nosso contexto de dados.

In [ ]:
def cont_reservas(dfDays):
    """
    Conta o número de reservas de todos os minutos de coleta para realizar uma média.
    
    Parameters
    ------------
    dfDays: Pandas dataframe
        Dados de todos os minutos de coleta a serem avaliados.
    
    Returns
    ----------
    media : Pandas dataframe
        Dados com a média das porcentagens para 24 horas.
    """
    
    # Coletando todos os minutos de captura
    datas = pd.to_datetime(dfDays['capture_time'])
    datas = pd.DataFrame(datas)

    dfReservas = pd.concat([dfTravels['car_id'], dfTravels['start'], dfTravels['end']], axis=1)

    # Ordenando os valores pelo tempo de inicio das reservas
    dfReservas = dfReservas.sort_values(by='start')

    cont_reservas = [0]*len(datas)

    # Percorrendo para cada id dos veículos
    for car in car_ids:
        print(car)
        reservas = dfReservas[dfReservas['car_id'] == car]
        timeline = []

        # Percorrendo todas as datas da coleta
        for i in range(len(datas)):
            data_timestamp = datas['capture_time'].iloc[i].timestamp()
            data = datas['capture_time'].iloc[i]
            estava_reservado = False

            # Percorrendo os intervalos de cada viagem
            for j in range(len(reservas)):
                # Verificando se o horário está entre o intervalo
                if (dfReservas['start'].iloc[j] <= data_timestamp <= dfReservas['end'].iloc[j]):
                    timeline.append(1)
                    estava_reservado = True
                    break

            # Se o veiculo não estava reservado em dado minuto
            if (not estava_reservado):
                timeline.append(0)

        # Somando os valores se estava ou não reservado
        cont_reservas = [x + y for x, y in zip(timeline, cont_reservas)]

    mean_reservas = [(x/len(car_ids))*100 for x in cont_reservas]
    reservas = pd.DataFrame()
    reservas['total_reserves'] = cont_reservas
    reservas['percentage'] = mean_reservas
    reservas['datetime'] = datas['capture_time']

    return reservas

In [ ]:
# Contando reservas durante os dias de coleta
dfR_Weekdays = cont_reservas(dfWeekdays)
dfR_Weekends = cont_reservas(dfWeekends)

In [ ]:
dfR_Weekends.to_csv('r_weekends_v2.csv', index=False, encoding='utf-8')
dfR_Weekdays.to_csv('r_weekdays_v2.csv', index=False, encoding='utf-8')

In [ ]:
# Leitura de dados já processados se necessário

# dfR_Weekdays =  pd.read_csv('r_weekdays_v2.csv')
# dfR_Weekends =  pd.read_csv('r_weekends_v2.csv')

# Formatando os dias para datetime

# dfR_Weekdays['datetime'] = pd.to_datetime(dfR_Weekdays['datetime'])
# dfR_Weekends['datetime'] = pd.to_datetime(dfR_Weekends['datetime'])

In [ ]:
# Plot da porcentagem de carros alocados em dias de semana
plt.plot(dfR_Weekdays['datetime'],dfR_Weekdays['percentage'])
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
# Plot da porcentagem de carros alocados em dias de semana
plt.plot(dfR_Weekdays['datetime'],dfR_Weekdays['percentage'])
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
# Plot da porcentagem de carros alocados em dias de semana
plt.plot(dfR_Weekends['datetime'],dfR_Weekends['percentage'])
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
# Fazendo a média das porcentagens de cada dia

# Dias de semana
dfR_Weekdays = dfR_Weekdays.sort_values(by='datetime')
dfR_Weekdays['capture_time'] = dfR_Weekdays['datetime']
dfmediaR_Weekdays = media(dfR_Weekdays)

# Ordenando pelo tempo
dfmediaR_Weekdays = dfmediaR_Weekdays.sort_values(by='time')

dfmediaR_Weekdays.to_csv('media_r_weekdays.csv', index=False, encoding='utf-8')

# Finais de semana
dfR_Weekends = dfR_Weekends.sort_values(by='datetime')
dfR_Weekends['capture_time'] = dfR_Weekends['datetime']
dfmediaR_Weekends = media(dfR_Weekends)

# Ordenando pelo tempo
dfmediaR_Weekends = dfmediaR_Weekends.sort_values(by='time')

dfmediaR_Weekends.to_csv('media_r_weekends.csv', index=False, encoding='utf-8')

## Plotagem final da porcentagem de carros reservados e ocupados

In [ ]:
import matplotlib

matplotlib.rc('font', size=12)

# Plot das porcentagens dos fins de semana
fig, (ax1, ax2) = plt.subplots(1, 2)

fig.set_size_inches(14,4.5)



# Curva dos carros andando

ax1.plot(range(len(mediaWeekdays['time'])),mediaWeekdays['mean'], label='Carros Ocupados')

# Curvas representando o intervalo de desvio padrão
ax1.plot(range(len(mediaWeekdays['time'])), mediaWeekdays['mean']+mediaWeekdays['std'], alpha=150, c='gray')
ax1.plot(range(len(mediaWeekdays['time'])), mediaWeekdays['mean']-mediaWeekdays['std'], alpha=150, c='gray')


# Curva dos carros reservados
ax1.plot(range(len(dfmediaR_Weekdays['time'])),dfmediaR_Weekdays['mean'], label='Carros Reservados', c='r', ls='--')

# Curvas representando o intervalo de desvio padrão
ax1.plot(range(len(dfmediaR_Weekdays['time'])), dfmediaR_Weekdays['mean']+dfmediaR_Weekdays['std'], alpha=150, c='#FA8072', ls='--')
ax1.plot(range(len(dfmediaR_Weekdays['time'])), dfmediaR_Weekdays['mean']-dfmediaR_Weekdays['std'], alpha=150, c='#FA8072', ls='--')


# Modificando os labels das horas e das porcentagens
ax1.xaxis.set_ticks(np.arange(0, 1441, 120))
ax1.yaxis.set_ticks(np.arange(0, 110, 10))

fig.canvas.draw()

labels = [item.get_text() for item in ax1.get_xticklabels()]
labels = range(0,26,2)

ax1.set_xticklabels(labels)

# Eixo y de 0 a 100%
ax1.set_ylim([0,100])

# Legendas e label dos eixos
ax1.legend(bbox_to_anchor=(0.01, 0.99), loc=2, borderaxespad=0.2)
ax1.set_ylabel('Percentual')
ax1.set_xlabel('Horário')




# # Curva dos carros andando
ax2.plot(range(len(mediaWeekends['time'])),mediaWeekends['mean'], label='Carros Ocupados')

# # Curvas representando o intervalo de desvio padrão
ax2.plot(range(len(mediaWeekends['time'])), mediaWeekends['mean']+mediaWeekends['std'], alpha=150, c='gray')
ax2.plot(range(len(mediaWeekends['time'])), mediaWeekends['mean']-mediaWeekends['std'], alpha=150, c='gray')


# # Curva dos carros reservados
ax2.plot(range(len(dfmediaR_Weekends['time'])),dfmediaR_Weekends['mean'], label='Carros Reservados', c='r', ls='--')

# # Curvas representando o intervalo de desvio padrão
ax2.plot(range(len(dfmediaR_Weekends['time'])), dfmediaR_Weekends['mean']+dfmediaR_Weekends['std'], alpha=150, c='#FA8072', ls='--')
ax2.plot(range(len(dfmediaR_Weekends['time'])), dfmediaR_Weekends['mean']-dfmediaR_Weekends['std'], alpha=150, c='#FA8072', ls='--')

# Modificando os labels das horas e das porcentagens
ax2.xaxis.set_ticks(np.arange(0, 1441, 120))
ax2.yaxis.set_ticks(np.arange(0, 110, 10))

fig.canvas.draw()

labels = [item.get_text() for item in ax2.get_xticklabels()]
labels = range(0,26,2)

ax2.set_xticklabels(labels)

# Eixo y de 0 a 100%
ax2.set_ylim([0,100])

# Legendas e label dos eixos
ax2.legend(bbox_to_anchor=(0.55, 0.99), loc=2, borderaxespad=0.1)
ax2.set_ylabel('Percentual')
ax2.set_xlabel('Horário')


plt.show()
plt.savefig('ViagensPorHoras_v2.pdf')